In [1]:
import re
import sys
import pandas as pd
from datetime import datetime


In [2]:
pd.options.display.max_columns= None
pd.options.display.max_colwidth= None
pd.options.display.max_rows = None

In [9]:
# ngram_analysis.py

from sklearn.feature_extraction.text import CountVectorizer
# from hazm import Normalizer, word_tokenize, Stemmer, Lemmatizer, stopwords_list

from connect_to_database_func import connect_db

# ---------------------------
# 1) Fetch comments from DB
# ---------------------------
def fetch_comments(sentiment=None, min_len=2, limit=None):
    """
    Fetch comments (id, title, grade, description, sentiment_result) from DB.
    Optionally filter by sentiment ('negative', 'positive', etc.) and non-empty text.
    """
    conn = connect_db()
    cur = conn.cursor()

    base = """
        SELECT id, title, grade, description, COALESCE(sentiment_result, '') as sentiment_result
        FROM comments
        WHERE description IS NOT NULL 
            AND is_repetitive = FALSE
           AND sentiment_score >0
    """
    args = []
    if sentiment:
        base += " AND lower(sentiment_result) = lower(%s)"
        args.append(sentiment)
    base += " ORDER BY id ASC"
    if limit:
        base += " LIMIT %s"
        args.append(limit)

    cur.execute(base, tuple(args))
    rows = cur.fetchall()
    cur.close()
    conn.close()

    df = pd.DataFrame(rows, columns=["id", "title", "grade", "description", "sentiment_result"])
    # # drop very short strings
    df = df[df["description"].str.len() >= min_len].reset_index(drop=True)
    return df






In [10]:
df = fetch_comments()

In [11]:
len(df)

27401

In [12]:
df.tail(5)

,id,title,grade,description,sentiment_result
27396,151158,مدیریت حساب‌ها,5,حساب به حساب,positive
27397,151159,دریافت تسهیلات,5,وام 1میلیونی رو یکجا تسویه کردم. نه وام 2 میلیونی جدید ثبت می‌شود و نه وام 1میلیونی مجدد. مدام خطای نامشخص می‌زند. لطفا پاسخ دهید,very negative
27398,151162,دریافت تسهیلات,1,باوجود تسویه یکجا وام یک میلیونی وگذشت یکماه ، error خطای نامشخص جهت دریافت جدید روی صفحه ظاهر میشود,negative
27399,151169,سایر,5,سلام واسه احراز هویت من هر کاری میکنم نمیشه خطا میزنه که تصویر با عکس اولیه تطابق ندارد در صورتی ک عکسی از من ثبت نشده میخام بدونم چجوری میتونم درسش کنم,negative
27400,151170,دریافت تسهیلات,5,احراز هویت برام ثبت نمیشه خطا میزنه ک با تصویر اولیه تطابق نداره چطوری میتونم‌این مشکل رو حل کنم,negative


In [13]:
df.head()

,id,title,grade,description,sentiment_result
0,1,پرداخت قبض,3,جایی نداره که من بنویسم این موبایل به نام چه کسی هست,no sentiment expressed
1,2,پرداخت قبض,3,به من نگفت که شماره تلفن را باید با کد شهر وارد کنی,negative
2,3,پرداخت قبض,3,قبض تلفن ثابت رو پرداخت کردم. به من نگفت با پیش شماره وارد کن.\n ذخیره هم نکرد قبض را,negative
3,4,پرداخت قبض,4,پس از پرداخت قبض و در حین باز شدن منوی تجربه، یک خطای انگلیسی در صفحه ظاهر شد و به صورت اتوماتیک بسته شد,negative
4,5,سایر,1,منوی حالات نمایش بصورت دوحالت تاریک و روشن میباشد و نیازی به گزینه پیش فرض ندارد.,negative


In [14]:
from preprocessing_main import preprocess

df["preprocessed_comments"]= df["description"].apply(lambda x: preprocess(x,convert_arabic_characters=True, remove_numbers=True, replace_multiple_spaces=True, convert_emojis=True, remove_diacritic=True))

In [15]:
df.head()

,id,title,grade,description,sentiment_result,preprocessed_comments
0,1,پرداخت قبض,3,جایی نداره که من بنویسم این موبایل به نام چه کسی هست,no sentiment expressed,جایی نداره که من بنویسم این موبایل به نام چه کسی هست
1,2,پرداخت قبض,3,به من نگفت که شماره تلفن را باید با کد شهر وارد کنی,negative,به من نگفت که شماره تلفن را باید با کد شهر وارد کنی
2,3,پرداخت قبض,3,قبض تلفن ثابت رو پرداخت کردم. به من نگفت با پیش شماره وارد کن.\n ذخیره هم نکرد قبض را,negative,قبض تلفن ثابت رو پرداخت کردم. به من نگفت با پیش شماره وارد کن. ذخیره هم نکرد قبض را
3,4,پرداخت قبض,4,پس از پرداخت قبض و در حین باز شدن منوی تجربه، یک خطای انگلیسی در صفحه ظاهر شد و به صورت اتوماتیک بسته شد,negative,پس از پرداخت قبض و در حین باز شدن منوی تجربه، یک خطای انگلیسی در صفحه ظاهر شد و به صورت اتوماتیک بسته شد
4,5,سایر,1,منوی حالات نمایش بصورت دوحالت تاریک و روشن میباشد و نیازی به گزینه پیش فرض ندارد.,negative,منوی حالات نمایش بصورت دوحالت تاریک و روشن میباشد و نیازی به گزینه پیش فرض ندارد.


In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mahdi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
import os
def load_stopwords(filepath="stopwords.txt"):
    """Load Persian stopwords from a text file (auto-clean quotes, commas, spaces)."""
    if not os.path.exists(filepath):
        print(f"⚠️ Warning: stopwords file not found at {filepath}")
        return set()

    stopwords = set()
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            word = (
                line.strip()
                .replace("'", "")   # remove single quotes
                .replace(",", "")   # remove commas
                .replace("\u200c", "")  # remove zero-width non-joiner
                .strip()
            )
            if word:
                stopwords.add(word)
    print(f"✅ Loaded {len(stopwords)} clean stopwords.")
    return stopwords


In [18]:
STOPWORDS = load_stopwords("stopwords.txt")

✅ Loaded 489 clean stopwords.


In [19]:
STOPWORDS

{'.',
 '{نمیشود',
 'آخر',
 'آخرش',
 'آخه',
 'آره',
 'آمد',
 'آمده',
 'آن',
 'آن ها',
 'آنها',
 'آیا',
 'احتمالا',
 'ادامه',
 'اره',
 'از',
 'ازش',
 'است',
 'استفاده',
 'اسم',
 'اش',
 'اصلا',
 'اضافه',
 'افتاد',
 'افتاده',
 'افزار',
 'الان',
 'البته',
 'ام',
 'اما',
 'امروز',
 'امشب',
 'ان',
 'انجام',
 'اند',
 'اندازه',
 'اندرکاران',
 'انقد',
 'انقدر',
 'انگار',
 'او',
 'اول',
 'اولین',
 'اومد',
 'اومده',
 'اون',
 'اون ها',
 'اونا',
 'اونایی',
 'اونجا',
 'اونم',
 'اونها',
 'اونی',
 'اين',
 'اکنون',
 'اگر',
 'اگه',
 'ای',
 'ایانیازبه',
 'ایشون',
 'ایم',
 'این',
 'این ها',
 'اینا',
 'اینجا',
 'اینجوری',
 'اینقدر',
 'اینم',
 'اینه',
 'اینها',
 'اینو',
 'اینکه',
 'با',
 'بار',
 'باز',
 'بازم',
 'بازگردند',
 'باسلام',
 'باش',
 'باشد',
 'باشم',
 'باشه',
 'باشی',
 'باشید',
 'باشیم',
 'باعث',
 'بالا',
 'بالاخره',
 'بانکها',
 'باید',
 'ببین',
 'ببینید',
 'بخاطر',
 'بخیر',
 'بد',
 'بدتر',
 'بدتره',
 'بدم',
 'بده',
 'بدون',
 'بدی',
 'بدید',
 'بدیم',
 'بر',
 'برا',
 'برابر',
 'برای',
 'برخی',
 'برد

In [20]:
# -----------------------------------
# 2) Preprocess & tokenize
# -----------------------------------
from nltk.tokenize import word_tokenize


def clean_and_tokenize(text: str):
    if not isinstance(text, str):
        return []

    # Use your custom preprocessing
    cleaned_text = preprocess(
        text,
        convert_farsi_numbers=True,
        convert_arabic_characters=True,
        remove_diacritic=True,
        remove_numbers=True,
        remove_punctuations=True,
        replace_multiple_spaces=True,
        remove_ha_suffix=True
    )

    # Use NLTK tokenizer (works fine for Persian with spacing)
    tokens = word_tokenize(cleaned_text)

    # Keep Persian words only (optional regex filter)
    tokens = [t for t in tokens if re.match(r"^[\u0600-\u06FF]+$", t)]

    # 🔹 Remove stopwords here
    tokens = [t.strip() for t in tokens if t.strip() not in STOPWORDS]
    
    return tokens



In [21]:
text = "این برنامه با بانک ملت خیلی خوب کار می‌کند"
tokens = clean_and_tokenize(text)
print(tokens)

['برنامه', 'بانک', 'خوب']


In [22]:
df["drop_stopword"] = df['description'].apply(lambda x: clean_and_tokenize(x))

In [23]:
df.head()

,id,title,grade,description,sentiment_result,preprocessed_comments,drop_stopword
0,1,پرداخت قبض,3,جایی نداره که من بنویسم این موبایل به نام چه کسی هست,no sentiment expressed,جایی نداره که من بنویسم این موبایل به نام چه کسی هست,"[بنویسم, موبایل, نام]"
1,2,پرداخت قبض,3,به من نگفت که شماره تلفن را باید با کد شهر وارد کنی,negative,به من نگفت که شماره تلفن را باید با کد شهر وارد کنی,"[نگفت, شماره, تلفن, کد, شهر]"
2,3,پرداخت قبض,3,قبض تلفن ثابت رو پرداخت کردم. به من نگفت با پیش شماره وارد کن.\n ذخیره هم نکرد قبض را,negative,قبض تلفن ثابت رو پرداخت کردم. به من نگفت با پیش شماره وارد کن. ذخیره هم نکرد قبض را,"[قبض, تلفن, ثابت, پرداخت, نگفت, شماره, ذخیره, نکرد, قبض]"
3,4,پرداخت قبض,4,پس از پرداخت قبض و در حین باز شدن منوی تجربه، یک خطای انگلیسی در صفحه ظاهر شد و به صورت اتوماتیک بسته شد,negative,پس از پرداخت قبض و در حین باز شدن منوی تجربه، یک خطای انگلیسی در صفحه ظاهر شد و به صورت اتوماتیک بسته شد,"[پرداخت, قبض, حین, منوی, تجربه, خطای, انگلیسی, صفحه, ظاهر, صورت, اتوماتیک, بسته]"
4,5,سایر,1,منوی حالات نمایش بصورت دوحالت تاریک و روشن میباشد و نیازی به گزینه پیش فرض ندارد.,negative,منوی حالات نمایش بصورت دوحالت تاریک و روشن میباشد و نیازی به گزینه پیش فرض ندارد.,"[منوی, حالات, نمایش, بصورت, دوحالت, تاریک, روشن, میباشد, نیازی, گزینه, فرض]"


In [24]:
# -----------------------------------
# 3) TF-IDF weighted n-grams
# -----------------------------------

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def extract_top_ngrams_tfidf(
    texts,
    ngram_range=(2, 3),
    top_k=30,
    min_df=3,
    max_df=0.6,
    max_features=30000
):
    """Extract top TF-IDF weighted n-grams"""
    vectorizer = TfidfVectorizer(
        tokenizer=clean_and_tokenize,
        preprocessor=lambda x: x,
        token_pattern=None,
        ngram_range=ngram_range,
        min_df=min_df,
        max_df=max_df,
        max_features=max_features
    )

    X = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = X.sum(axis=0).A1

    df_tfidf = pd.DataFrame({
        "ngram": feature_names,
        "tfidf": np.round(tfidf_scores,4)
    }).sort_values("tfidf", ascending=False).head(top_k).reset_index(drop=True)

    df_tfidf["n"] = df_tfidf["ngram"].str.count(" ") + 1
    # df_tfidf["n"] = df_tfidf["ngram"].str.count(" ") + 1
    print(df_tfidf.groupby("n").head(10))
    return df_tfidf



In [26]:
# 4) Group sentiments (3 categories)
# ---------------------------------
def group_sentiments(df):
    df = df.copy()
    df["sentiment_group"] = "neutral"  # default group

    df.loc[df["sentiment_result"].str.lower().isin(["negative", "very negative"]), "sentiment_group"] = "negative"
    df.loc[df["sentiment_result"].str.lower().isin(["positive", "very positive"]), "sentiment_group"] = "positive"
    df.loc[df["sentiment_result"].str.lower().isin(["neutral", "mixed", "no sentiment expressed"]), "sentiment_group"] = "neutral"

    return df



In [27]:
import nltk

# Add local path in case it's not found
nltk.data.path.append("/home/mahdi/nltk_data")

# Ensure both punkt and punkt_tab are available
for pkg in ["punkt", "punkt_tab"]:
    try:
        nltk.data.find(f"tokenizers/{pkg}")
    except LookupError:
        nltk.download(pkg, quiet=True)


In [28]:
# ---------------------------------
# 5) Main analysis pipeline
# ---------------------------------
import os

def main(limit=None):
    print("Fetching comments from database...")
    df = fetch_comments(limit)
    if df.empty:
        print("No comments found.")
        return

    df = group_sentiments(df)
    print(f"Total comments: {len(df)}")

    # ----- ALL comments
    print("\n🔹 Top n-grams (ALL):")
    df_all_3 = extract_top_ngrams_tfidf(df["description"].tolist(), ngram_range=(3, 3))
    df_all_2 = extract_top_ngrams_tfidf(df["description"].tolist(), ngram_range=(2, 2))

    # Label each type
    df_all_2["type"] = "bigram"
    df_all_3["type"] = "trigram"

    # Combine
    df_all = pd.concat([df_all_2, df_all_3], ignore_index=True)
    print(df_all)

    # ----- NEGATIVE
    df_neg = df[df["sentiment_group"] == "negative"]
    if not df_neg.empty:
        print("\n🔴 Top n-grams (NEGATIVE group):")

        # Extract both bigrams and trigrams
        df_neg_2 = extract_top_ngrams_tfidf(df_neg["description"].tolist(), ngram_range=(2, 2))
        df_neg_3 = extract_top_ngrams_tfidf(df_neg["description"].tolist(), ngram_range=(3, 3))

        # Label each
        df_neg_2["type"] = "bigram"
        df_neg_3["type"] = "trigram"

        # Combine
        df_neg_tfidf = pd.concat([df_neg_2, df_neg_3], ignore_index=True)

        print(df_neg_tfidf)
    else:
        df_neg_tfidf = pd.DataFrame()


    # ----- POSITIVE
    df_pos = df[df["sentiment_group"] == "positive"]
    if not df_pos.empty:
        print("\n🟢 Top n-grams (POSITIVE group):")

        df_pos_2 = extract_top_ngrams_tfidf(df_pos["description"].tolist(), ngram_range=(2, 2))
        df_pos_3 = extract_top_ngrams_tfidf(df_pos["description"].tolist(), ngram_range=(3, 3))

        df_pos_2["type"] = "bigram"
        df_pos_3["type"] = "trigram"

        df_pos_tfidf = pd.concat([df_pos_2, df_pos_3], ignore_index=True)

        print(df_pos_tfidf)
    else:
        df_pos_tfidf = pd.DataFrame()


    # ----- NEUTRAL
    df_neu = df[df["sentiment_group"] == "neutral"]
    if not df_neu.empty:
        print("\n⚪ Top n-grams (NEUTRAL group):")

        df_neu_2 = extract_top_ngrams_tfidf(df_neu["description"].tolist(), ngram_range=(2, 2))
        df_neu_3 = extract_top_ngrams_tfidf(df_neu["description"].tolist(), ngram_range=(3, 3))

        df_neu_2["type"] = "bigram"
        df_neu_3["type"] = "trigram"

        df_neu_tfidf = pd.concat([df_neu_2, df_neu_3], ignore_index=True)

        print(df_neu_tfidf)
    else:
        df_neu_tfidf = pd.DataFrame()


    # ----- Save to CSVs
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)

    df_all.to_csv(os.path.join(output_dir, "tfidf_all-0.2.csv"), index=False)
    if not df_neg_tfidf.empty:
        df_neg_tfidf.to_csv(os.path.join(output_dir, "tfidf_negative-0.2.csv"), index=False)
    if not df_pos_tfidf.empty:
        df_pos_tfidf.to_csv(os.path.join(output_dir, "tfidf_positive-0.2.csv"), index=False)
    if not df_neu_tfidf.empty:
        df_neu_tfidf.to_csv(os.path.join(output_dir, "tfidf_neutral-0.2.csv"), index=False)

    print("\n✅ TF-IDF n-gram analysis completed. CSV files generated.")




In [29]:
if __name__ == "__main__":
    import nltk
    nltk.download("punkt", quiet=True)
    sys.exit(main(limit=None))  # set limit=1000 for faster testing

Fetching comments from database...
Total comments: 27401

🔹 Top n-grams (ALL):
                ngram     tfidf  n
0     احراز هویت مشکل  123.4783  3
1  نمیتونم احراز هویت  120.7806  3
2      احراز هویت خطا  101.9203  3
3    احراز هویت تایید   56.3502  3
4     قسمت احراز هویت   54.4215  3
5  شماره سریال اشتباه   50.6123  3
6    مرحله احراز هویت   47.1817  3
7     احراز هویت قبول   42.1447  3
8     مشکل احراز هویت   33.7422  3
9      عالی عالی عالی   30.4637  3
           ngram      tfidf  n
0     احراز هویت  1151.9571  2
1     انتقال وجه   172.4246  2
2    درخواست وام   142.2023  2
3      عالی سریع   121.2961  2
4      عالی عالی   109.6629  2
5  نمیتونم احراز   106.8122  2
6     اهراز هویت   105.1708  2
7      هویت مشکل   103.9717  2
8     همراه بانک   100.0157  2
9      کارت کارت    91.7484  2
                 ngram      tfidf  n     type
0           احراز هویت  1151.9571  2   bigram
1           انتقال وجه   172.4246  2   bigram
2          درخواست وام   142.2023  2   bigram
3          

SystemExit: 

/home/mahdi/Sentiment_Analysis_Dima/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Summarization

In [30]:
def prepare_comments_for_summary(df, max_comments=50):
    texts = (
        df["description"]
        .dropna()
        .astype(str)
        .str.strip()
        .tolist()
    )

    # Keep reasonable-length comments
    texts = [t for t in texts if len(t) > 20]

    return texts[:max_comments]


In [31]:
def extractive_summary(texts, top_k=5):
    """
    Simple extractive summary: returns top K representative comments
    """
    return "\n- " + "\n- ".join(texts[:top_k])


In [33]:
df = group_sentiments(df)
neg_df = df[df["sentiment_group"] == "negative"]
neg_texts = prepare_comments_for_summary(neg_df)

summary_negative = extractive_summary(neg_texts)

print("🔴 Negative Comments Summary:")
print(summary_negative)


🔴 Negative Comments Summary:

- به من نگفت که شماره تلفن را باید با کد شهر وارد کنی
- قبض تلفن ثابت رو پرداخت کردم. به من نگفت با پیش شماره وارد کن.
 ذخیره هم نکرد قبض را
- پس از پرداخت قبض و در حین باز شدن منوی تجربه، یک خطای انگلیسی در صفحه ظاهر شد و به صورت اتوماتیک بسته شد
- منوی حالات نمایش بصورت دوحالت تاریک و روشن میباشد و نیازی به گزینه پیش فرض ندارد.
- تعداد ایده و نظرات کاربر مشخص باشه که کاربر بدونه آیا واقعا ایده و نظرش ثبت شده یا خیر. یک حس تعاملی بهتری به کاربر خواهد داد.
 
 ضمنا در ثبت ایده ها هنوز تجربه ای شکل نگرفته که انرا ستاره دار کند و واقعا ابهام داره که باید اینجا امتیاز به چه چیزی بدهد


In [34]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="persiannlp/mt5-base-parsinlu-summarization",
    device=-1
)

def abstractive_summary(texts, max_chars=3000):
    joined = " ".join(texts)
    joined = joined[:max_chars]  # IMPORTANT

    out = summarizer(
        joined,
        max_length=150,
        min_length=60,
        do_sample=False
    )
    return out[0]["summary_text"]


/home/mahdi/Sentiment_Analysis_Dima/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: persiannlp/mt5-base-parsinlu-summarization is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [45]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def summarize_extractive_tfidf(
    comments: list[str],
    top_k: int = 7,
    min_len: int = 20,
    max_comments: int = 200,
):
    # Clean list
    texts = [c.strip() for c in comments if isinstance(c, str) and len(c.strip()) >= min_len]
    if not texts:
        return []
    texts = texts[:max_comments]

    n_docs = len(texts)

    # If too few docs, just return first top_k (or longest) safely
    if n_docs < 3:
        texts_sorted = sorted(texts, key=len, reverse=True)
        return texts_sorted[:min(top_k, len(texts_sorted))]

    # Adaptive df thresholds
    min_df = 2
    max_df = 0.85  # a bit more permissive

    # Ensure max_df won't imply fewer docs than min_df
    # max_df_docs = floor(max_df * n_docs) must be >= min_df
    max_df_docs = int(np.floor(max_df * n_docs))
    if max_df_docs < min_df:
        max_df = 1.0  # disable max_df filtering for small sets

    vec = TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=min_df,
        max_df=max_df,
    )
    X = vec.fit_transform(texts)

    centroid = X.mean(axis=0)
    centroid = np.array(centroid).ravel().reshape(1, -1)  # safe dense 2D
    scores = cosine_similarity(X, centroid).ravel()

    top_idx = np.argsort(scores)[::-1][:min(top_k, len(texts))]
    return [texts[i] for i in top_idx]


In [46]:
neg_comments = df[df["sentiment_group"] == "negative"]["description"].tolist()
picked = summarize_extractive_tfidf(neg_comments, top_k=7)

print("🔴 Negative summary (representative comments):")
for t in picked:
    print("-", t)


🔴 Negative summary (representative comments):
- ضمن تشکر بابت ایجاد اپ، چند مورد به شرح زیر قابل طرح است:
 - مدت زمانی که طول میکشد تا اپ از اکانت خارج شود خیلی کوتاه است و وسط یکی از تراکنش ها این اتفاق افتاد برای من
 - در بخش "حساب" موجودی هیچکدام از حساب های من حتی باوجود رفرش کردن نمایش داده نمیشود!
 - فیلد تکمیل کننده تاریخ انقضا در بخش انتقال وجه، کارت به کارت، بهتر است با اسلش یا علامت دیگری جدا شود که کاربر دچار اشتباه نشود که اول ماه را بزند یا سال را.
 باتشکر
- ۱. در قسمت ثبت نام اولیه، بهتره کلمه بایومتریک، به بیومتریک اصلاح بشه
 ۲. اون قسمتی که باید نام کاربری دلخواه بزنیم، علامت سوال کار نمیکنه، به مشتری بهتره بگیم نام کاربری باید چه ویژگی هایی داشته باشه، مثلا seyed_m_rahim رو قبول نکرد و خطا میداد
 ۳. در همین قسمت ثبت ایده نمیشه متن رو کپی کرد سلکت هم مشکل داره
 ۴. در تسهیلات میکرولون، خطای پردازش داده وجود داره
 ۵. کاور های تبلیغی قسمت هوم برنامه کیفیت لازمه رو نداره
 ۶. در قسمت احراز هویت ، سایت مربوطه ekyc، ui مناسب و هم راستا با اپلیکیشن نداره
 ۷‌. در قسمت بازخورد نو

In [47]:
pos_comments = df[df["sentiment_group"] == "positive"]["description"].tolist()
picked = summarize_extractive_tfidf(pos_comments, top_k=7)

print("🔴 Positive summary (representative comments):")
for t in picked:
    print("-", t)


🔴 Positive summary (representative comments):
- سلام وقت بخیر
رضا اسدی همکار شما هستم
چنتا پیشنهاد داشتم
1. برای اطلاعات اعتباری اشخاص حقیقی هم بتونید داخل همین نرم افزار مشتری درخواست بده واقعا فوق العاده میشه
کارشناس هم مدارک رو از این نرم افزار دریافت میکنه
و اصالت و سرعت بیشتری هم داره و رضایت مشتری هم در بر میگیره
2.وام های پذیرندگان هست که حتما براش برنامه ریزی کردن و لازمه اون حذف کاغذبازی های مختلف در کنار منافع بانک هست
که اگه اونم با نوآوری وارد این آپ بشه 
3.سوم اینکه درخواست مشتری برای قانون186مالیلت در اپلیکیشن ثبت شود و وام پذیرندگان با مبلغ مشخص در اینجا ثبت شود و کارهای سیستمی و ادامه کار از ایجا استخراج شود
4. درخواست ضمانت های درخواستی برای اشخاص حقیقی و حقوقی داخل این نرم افزار هست
به شکلی سهم از بازار بانک رو به شکلی قابل توجه بالا خواهد برد
5.ایجاد صندوقی به نام قلک برای افراد زیر 18سال  و در مقابل به آنها تیکت های تخفیف فروشگاهی اینترنتی و حضوری پذیرندگان بانک ملت رو ارائه بدهیم یعنی هم برای مشتریان خودمون هم مشتری می‌فرستیم و هم هزینه ای بابت این کار فوق‌العاده پ